In [16]:
load('../Sage Code/sidon_cryptosystem.sage')

In [17]:
##Implementation of the first minrank attack in sidon cryptosystem paper
##We first define the sidon space we are trying to break
q = 7
basefield = GF(q)
k = 3
r = 2
rk = r*k
y, F, F_r, d, c = ConstructSidon2k(q, k)
##Construct the attacker's F_r
y2, F2, F_r2, d2, c2 = ConstructSidon2k(q, k)
matrixList, sidonbasis, mult_table, F_r_basis , origbasis = publicKey(y,q,F,F_r)

[6 4 5 6 2 6]
[3 1 2 3 3 6]
[6 1 5 1 6 2]
[4 0 5 2 6 6]
[6 3 1 1 5 2]
[6 2 6 2 2 4]


In [18]:
##Construct the attacker's basis of F and F_r
iterations = 0
sidonbasis = Matrix(basefield, k, lambda i,j: basefield.random_element()*0)
##basis of F
while sidonbasis.is_invertible() == False and iterations < 500:
    element = F.random_element()
    sidonbasis = Matrix([list(vector(element^(q^i))) for i in range(k)])
    iterations += 1
v = [F2(list(sidonbasis[i])) for i in range(k)]
sidonbasis = [j*y2*y2^-1 for  j in v]
sidonbasismat = Matrix([convertToLong(i) for i in sidonbasis])
##Extend F to F_r
F_r2basis = Matrix(basefield, k, lambda i,j: basefield.random_element()*0)
while F_r2basis.is_invertible() == False and iterations < 500:
    F_r2basis = sidonbasismat
    stackmat = Matrix(basefield, rk - k, rk, lambda i, j: basefield.random_element())
    F_r2basis = F_r2basis.stack(stackmat)
    iterations += 1
    print(F_r2basis.rank(), F_r2basis.nrows(), F_r2basis.ncols())

6 6 6


In [19]:
F_r2basisels =  [convertFromLong(F_r2basis[i], F2, F_r2) for i in range(rk)]

In [20]:
##define variables
g = ['g' + str(i) for i in range(rk)]

In [21]:
##create a polynomial ring over the variables in order to do symbolic calculations 
b = [["b" + str(j) + str(i) for i in range(rk)] for j in range(rk)]
u = [["u" + str(j) + str(i) for i in range(k) for j in range(k)]]
bvec = []
uvec = []
for i in b: 
    bvec += i
for i in u: 
    uvec += i
R = PolynomialRing(F_r2, g + bvec + uvec)
indeterminates = R.gens()
R.inject_variables()

for i in range(rk): 
    g[i] = R(g[i])
for i in range(k*k): 
    uvec[i] = R(uvec[i])
for i in range(rk*rk):
    bvec[i] = R(bvec[i])
y2 = vector(g).row()*vector(F_r2basisels).column()
F_r2_q = vector([R(i^q) for i in F_r2basisels])

Defining g0, g1, g2, g3, g4, g5, b00, b01, b02, b03, b04, b05, b10, b11, b12, b13, b14, b15, b20, b21, b22, b23, b24, b25, b30, b31, b32, b33, b34, b35, b40, b41, b42, b43, b44, b45, b50, b51, b52, b53, b54, b55, u00, u10, u20, u01, u11, u21, u02, u12, u22


In [22]:
v = [] 
for i in range(k):
    v += vector(uvec[i*k:(i+1)*k]).row()*vector(F_r2basisels[0:k]).column() + y2*vector(uvec[i*k:(i+1)*k]).row()*vector(F_r2_q[0:k]).column() 

In [23]:
B = [0 for i in range(rk)]
for i in range(rk): 

    for j in range(rk):
        B[i] += R(bvec[i*rk + j])*R(F_r2_q[j])

In [24]:
bvec[0:rk]

[b00, b01, b02, b03, b04, b05]

In [25]:
v_mult_table =[[v[i]*v[j] for i in range(len(v))] for j in range(len(v))]

In [26]:
B_table = [[0 for i in range(k)] for j in range(k)]
eq_table = [[0 for i in range(k)] for j in range(k)]
for i in range(k): 
    for j in range(k): 
        sum1 = 0
        for l in range(rk): 
            sum1 += R(matrixList[l][i][j])*B[l]
        B_table[i][j] = sum1  
        eq_table[i][j] = R(v_mult_table[i][j])  - sum1

In [27]:
(convertToLong(F_r2(list(eq_table[0][0])[0][0]))*F_r2basis.inverse()*g0).parent()

Ambient free module of rank 6 over the integral domain Multivariate Polynomial Ring in g0, g1, g2, g3, g4, g5, b00, b01, b02, b03, b04, b05, b10, b11, b12, b13, b14, b15, b20, b21, b22, b23, b24, b25, b30, b31, b32, b33, b34, b35, b40, b41, b42, b43, b44, b45, b50, b51, b52, b53, b54, b55, u00, u10, u20, u01, u11, u21, u02, u12, u22 over Univariate Quotient Polynomial Ring in a over Finite Field in z of size 7^3 with modulus a^2 + 6*z^2 + 6

In [28]:
vectorized_eq_table = [[0 for i in range(k)] for j in range(k)]
for i in range(k): 
    for j in range(k): 
        if i > j: 
            continue
        else: 
            eqvec = list(eq_table[i][j])
            sumvec = vector([0 for l in range(rk)])
            for term in eqvec: 
                coeff = term[0]
                monomial = term[1]
                coeff = convertToLong(F_r2(coeff))
                rep = coeff*F_r2basis.inverse()
                sumvec += rep*monomial
        vectorized_eq_table[i][j] = sumvec

In [32]:
vectorized_eq_table[0][1][2]

5*g0^2*u00*u01 - g0*g1*u00*u01 - g1^2*u00*u01 + 5*g1*g2*u00*u01 + 4*g2^2*u00*u01 + 4*g0*g3*u00*u01 + g1*g3*u00*u01 + 4*g2*g3*u00*u01 + 5*g3^2*u00*u01 + g0*g4*u00*u01 + 2*g1*g4*u00*u01 - g2*g4*u00*u01 + 2*g3*g4*u00*u01 + 4*g0*g5*u00*u01 + g1*g5*u00*u01 + 2*g3*g5*u00*u01 + g4*g5*u00*u01 + g5^2*u00*u01 - g1^2*u10*u01 - g0*g2*u10*u01 + g1*g2*u10*u01 + g2^2*u10*u01 + 4*g1*g3*u10*u01 + 2*g2*g3*u10*u01 - g3^2*u10*u01 - g1*g4*u10*u01 + 2*g3*g4*u10*u01 + 4*g4^2*u10*u01 + g2*g5*u10*u01 + 4*g3*g5*u10*u01 + 3*g4*g5*u10*u01 - g5^2*u10*u01 + 3*g0*g1*u20*u01 + 3*g1^2*u20*u01 + 3*g2^2*u20*u01 + g0*g3*u20*u01 + 4*g1*g3*u20*u01 + g3^2*u20*u01 + 5*g0*g4*u20*u01 + g1*g4*u20*u01 + 4*g3*g4*u20*u01 - g4^2*u20*u01 + 4*g1*g5*u20*u01 + 5*g3*g5*u20*u01 + 2*g5^2*u20*u01 - g1^2*u00*u11 - g0*g2*u00*u11 + g1*g2*u00*u11 + g2^2*u00*u11 + 4*g1*g3*u00*u11 + 2*g2*g3*u00*u11 - g3^2*u00*u11 - g1*g4*u00*u11 + 2*g3*g4*u00*u11 + 4*g4^2*u00*u11 + g2*g5*u00*u11 + 4*g3*g5*u00*u11 + 3*g4*g5*u00*u11 - g5^2*u00*u11 - g0*g1*u10*u11 

In [2]:
##Algebraic attack implementation is also here
def makePublicKey(q,k):
    basefield = GF(q)
    rk = 2*k
    y, F, F_r, d, c = ConstructSidon2k(q, k)
    ##Construct the attacker's F_r
    y2, F2, F_r2, d2, c2 = ConstructSidon2k(q, k)
    matrixList, sidonbasis, mult_table, F_r_basis , origbasis = publicKey(y,q,F,F_r)
    return matrixList
def algebraicAttack(q,k,a,b, matrixList):
    rhs = [a.row()*i*b.column() for i in matrixList]
    R_ = PolynomialRing(GF(q), ['x' + str(i) for i in range(2*k)] + ['xn'], order = "degrevlex")
    indeterminates = R_.gens()[:-1]
    xn = R_.gens()[-1]
    R_.inject_variables()
    a1 = vector(indeterminates[0:k])
    b1 = vector(indeterminates[k:])
    system = [a1.row()*i*b1.column() for i in matrixList]
    system = [system[i][0][0] - rhs[i][0][0]*xn^2 for i in range(len(system))]
    I = ideal(system)
    gb = I.groebner_basis()
    I = ideal(gb)
    I  = I.subs(x0 = 1)
    I = I.subs(xn= 1)
    R_  = R_.remove_var(xn)
    R_  = R_.remove_var(x0)
    gens = [R_(i) for i in I.gens()]
    I = ideal(gens)
    I = I.groebner_basis()
    I = I.ideal()
    return I, matrixList

In [ ]:
q = 7
k = 4
a = vector([5,1,2,1])
b = vector([1,2,1,4])
makePublicKey(q,k)
c, d = algebraicAttack(q,k,a,b, matrixList)

In [ ]:
a1 = vector(indeterminates[0:k])
b1 = vector(indeterminates[k:])
system = [a1.row()*i*b1.column() for i in matrixList]

In [ ]:
c

In [ ]:
R_.is_commutative()

In [ ]:
system[0][0][0].parent()

In [3]:
matrixList = makePublicKey(7,7)
a = vector([1,2,3,4,5,6,0])
b = vector([0,1,2,3,4,5,6])
i1, m = algebraicAttack(7,7,a,b,matrixList)

[3 3 4 0 6 4 4 2 6 4 5 3 0 0]
[0 4 5 4 2 1 6 5 0 5 2 3 6 3]
[0 1 2 6 6 5 6 1 6 1 1 1 4 1]
[3 0 2 1 6 1 6 0 2 5 6 0 3 1]
[0 4 5 0 0 2 2 3 3 3 3 3 4 6]
[1 6 4 2 2 1 3 6 4 1 0 6 3 0]
[6 6 6 4 1 1 2 4 3 6 4 3 2 5]
[5 1 3 6 5 5 0 3 0 5 0 2 1 0]
[0 6 6 3 3 1 1 4 0 6 6 6 2 4]
[3 4 1 2 3 1 2 1 5 2 3 6 1 2]
[0 6 1 2 3 6 5 2 3 3 5 4 6 4]
[5 4 6 5 2 4 0 4 6 3 6 5 5 5]
[4 1 6 3 6 1 6 5 4 2 1 6 0 0]
[1 1 5 0 2 3 6 1 4 6 1 2 3 4]
Defining x0, x1, x2, x3, x4, x5, x6, x7, x8, x9, x10, x11, x12, x13, xn


KeyboardInterrupt: 

In [ ]:
I  = I.subs(xn= 1, x0 = 1)

In [ ]:
I.dimension()

In [ ]:
I = I.groebner_basis()

In [ ]:
I = ideal(I)

In [ ]:
v = I.subs(x0 = 5, x1 = 1, x2 = 2)
v = v.subs(x3 = 2,x4  = 1, x5 = 1)

In [ ]:
v.variety()

In [ ]:
v.dimension()

In [ ]:
I.dimension()

In [ ]:
I

In [ ]:
I.remove_variable(xn)

In [ ]:
R_  = R_.remove_var(xn)

In [ ]:
gens = [R_(i) for i in I.gens()]

In [ ]:
v = ideal(gens)

In [ ]:
v.dimension()

In [ ]:
v = v.subs(x0 = 5)

In [ ]:
R_  = R_.remove_var(x0)

In [ ]:
gens = [R_(i) for i in v.gens()]

In [ ]:
v.variety()

In [3]:
def algebraicAttackSystem(q,k,a,b, matrixList):
    rhs = [a.row()*i*b.column() for i in matrixList]
    R_ = PolynomialRing(GF(q), ['x' + str(i) for i in range(2*k)] + ['xn'], order = "degrevlex")
    indeterminates = R_.gens()[:-1]
    xn = R_.gens()[-1]
    R_.inject_variables()
    a1 = vector(indeterminates[0:k])
    b1 = vector(indeterminates[k:])
    system = [a1.row()*i*b1.column() for i in matrixList]
    system = [system[i][0][0] - rhs[i][0][0]*xn^2 for i in range(len(system))]
    return system, R_
# implementation of Arri's algorithm to compute a Groebner basis
# "The F5 Criterion revised"
# paper submitted to JSC in 2009, accepted conditionally in 2010 subject to revision,
# resubmitted 2010
# http://arxiv.org/abs/1012.3664

# usage: f5_arri(F) where F is a list of polynomials
# result: groebner basis of F, with signatures
# example:
#   sage: R = PolynomialRing(GF(32003),x,5)
#   sage: I = sage.rings.ideal.Cyclic(R,5).homogenize()
#   sage: F = list(I.gens())
#   sage: B = f5_arri(F)
#   >> There were 0 reductions to zero.
#   sage: len(B)
#   >> 166
#   sage: F.reverse()
#   sage: B = f5_arri(F)
#   >> There were 0 reductions to zero.
#   sage: len(B)
#   >> 39
# as with all incremental algorithms, efficiency depends on the ordering of F

verbose_zero_reduction = 0

verbose_generation = 1
verbose_reduction = 1

verbose_rejection = 1

# polysort
# used to sort polynomials by increasing signature
# inputs: (f,S(f)) and (g,S(g)), labeled polynomials
# outputs: 1 if (f,S(f)) > (g,S(g))
#        -1 if (f,S(f)) < (g,S(g))
#         0 if (f,S(f)) = (g,S(g))
def polysort(f,g):
  fp,Sf = f
  gp,Sg = g
  sigord = sigsort(Sf,Sg)
  if sigord != 0:
    return sigord
  else:
    if fp.lm() > gp.lm():
      return 1
    else:
      return -1
  return 0

# sigsort
# used to sort signatures
# inputs: S(f) and S(g), labeled polynomials
# outputs: 1 if S(f) > S(g)
#        -1 if S(f) < S(g)
#         0 if S(f) = S(g)
def sigsort(s1,s2):
  t1,i1 = s1
  t2,i2 = s2
  if i1 < i2:
    return 1
  if i1 > i2:
    return -1
  if t1 > t2:
    return 1
  if t1 < t2:
    return -1
  return 0

# is_primitive
# tests whether a polynomial is primitive
# inputs: polynomial f, S(f), basis G
# outputs: whether uf*f is primitive
def is_primitive(f, Sf, G):
  R = f.parent()
  for (h,Sh) in G:
    if h != f and Sh[1] == Sf[1] and R.monomial_divides(Sh[0],Sf[0]):
      t = R.monomial_quotient(Sf[0],Sh[0])
      if t*h.lm() == f.lm():
        return False
  return True

# prune_rewritables
# removes rewritable elements from B
# this is NOT the same as Faugere's rewritable algorithm;
# see the paper for details
# inputs: polynomial ring R, wokring basis G, S-polynomials B
# outputs: (probably) smaller B
def prune_rewritables(R,G,B):
  Bnew = set()
  for (f,Sf) in B:
    rewritable = False
    t = f.lm()
    for (g,Sg) in B:
      if Sg[1] == Sf[1] and R.monomial_divides(Sg[0],Sf[0]):
        u = R.monomial_quotient(Sf[0],Sg[0])
        if u*g.lm() < t:
          rewritable = True
          continue
    if not rewritable:
      for (g,Sg) in G:
        if Sg[1] == Sf[1] and R.monomial_divides(Sg[0],Sf[0]):
          u = R.monomial_quotient(Sf[0],Sg[0])
          if u*g.lm() < t:
            rewritable = True
            continue
      if not rewritable:
        Bnew.add((f,Sf))
  return Bnew

# is_normal
# completes test whether a critical pair is a normal pair in Arri's definition
# (beyond inequality of signature)
# inputs: polynomials f and g, S(f), S(g) monomials uf and ug, basis G
# outputs: whether (f,g) is a normal pair
def is_normal(f, Sf, uf, G, B):
  R = f.parent()
  # Faugere's normal?
  if not any(Sh[1]>Sf[1] and R.monomial_divides(h.lm(),uf*Sf[0]) for (h,Sh) in G):
    # primitive?
    return is_primitive(f,Sf,G) and is_primitive(f,Sf,B)
    #return True
  # not even Faugere's normal, let alone primitive
  return False

# Sreduce
# inputs: polynomial f, S(f), basis G, rewritings RW
# outputs: S-irreducible form of f
def Sreduce(f,Sf,G,B):
  R = f.parent()
  reduced = True
  while reduced:
    reduced = False
    for (g,Sg) in G:
      if R.monomial_divides(g.lm(),f.lm()):
        t = R.monomial_quotient(f.lm(),g.lm())
        if sigsort(Sf,(t*Sg[0],Sg[1])) == 1:
          if is_normal(g, Sg, t, G, B) and is_primitive(g, Sg, G) and is_primitive(g, Sg, B):
            reduced = True
            f -= t*g
            if f == 0:
              return f
            f *= f.lc()**(-1)
          else:
            verbose("reduction of "+str(Sf)+" by "+str(Sg)+" rejected: not normal",level=verbose_rejection)
        else:
          verbose("reduction of "+str(Sf)+" by "+str(Sg)+" rejected: too large",level=verbose_rejection)
  return f

# UpdatePairs
# inputs: basis G, known new polynomials B, polynomial f, S(f)
# outputs: new S-polynomials
def UpdatePairs(G,B,f,Sf):
  Bnew = set()
  if is_primitive(f,Sf,G) and is_primitive(f,Sf,B):
    R = f.parent()
    for (g,Sg) in G:
      tf = f.lm()
      tg = g.lm()
      t = tf.lcm(tg)
      uf = R.monomial_quotient(t,tf)
      ug = R.monomial_quotient(t,tg)
      # normal pair?
      siguf = (uf*Sf[0],Sf[1])
      sigug = (ug*Sg[0],Sg[1])
      if siguf != sigug:
        if is_normal(f,Sf,uf,G,B) and is_normal(g,Sg,ug,G,B):
          if sigsort(siguf,sigug) == 1:
            sig = siguf
          else:
            sig = sigug
          verbose("generating "+str(sig)+" from "+str(Sf)+" and "+str(Sg),level=verbose_generation)
          S = uf*f - ug*g
          S *= S.lc()**(-1)
          Bnew.add((S,sig))
        else:
          verbose("pair "+str(Sf)+", "+str(Sg)+" rejected: not normal",level=verbose_rejection)
      else:
        verbose("pair "+str(Sf)+", "+str(Sg)+" rejected: equal signature",level=verbose_rejection)
  return Bnew
  
# prune_not_normal
# prunes polynomials of signature that are not normal
# inputs list of polynomials B, list of polynomials G
# outputs: pruned version of B
def prune_not_normal(B,G):
  Bnew = set()
  while len(B) != 0:
    f,Sf = B.pop()
    if is_normal(f,Sf,Integer(1),G,Bnew.union(B)):
      Bnew.add((f,Sf))
  return list(Bnew)

# f5_arri
# compute groebner basis
# inputs: list of polynomials F
# outputs: groebner basis (we hope) of ideal of F
def f5_arri(F):
  L = set()
  G = set()
  R = F[0].parent()
  B = [(F[i]*F[i].lc()**(-1),(R(1),i)) for i in range(len(F))]
  while len(B) != 0:
    Bnew = set()
    for (f,Sf) in B:
      if not any(Sg[1]==Sf[1] and R.monomial_divides(Sg[0],Sf[0]) for Sg in L):
        Bnew.add((f,Sf))
    B = Bnew
    #i = len(B)
    B = prune_rewritables(R,G,B)
    #if len(B) < i:
    #  print "pruned", i - len(B)
    B = prune_not_normal(B,G)
    
    B.sort(key =  cmp_to_key(polysort))
    if len(B)!= 0:
        f,Sf = B.pop(0)
    while len(B) != 0 and B[0][1] == Sf:
      B.pop(0)
    f = Sreduce(f,Sf,G,B)
    verbose("reduced "+str(Sf)+" to "+str(f.lm()),level=verbose_reduction)
    if f != 0:
      if is_primitive(f,Sf,G) and is_primitive(f,Sf,B):
        B.extend(UpdatePairs(G,B,f,Sf))
      G.add((f,Sf))
    else:
      L.add(Sf)
      verbose("zero reduction: "+str(Sf),level=verbose_zero_reduction)
  print("There were", len(L), "reductions to zero.")
  return G
def cmp_to_key(mycmp):
    'Convert a cmp= function into a key= function'
    class K:
        def __init__(self, obj, *args):
            self.obj = obj
        def __lt__(self, other):
            return mycmp(self.obj, other.obj) < 0
        def __gt__(self, other):
            return mycmp(self.obj, other.obj) > 0
        def __eq__(self, other):
            return mycmp(self.obj, other.obj) == 0
        def __le__(self, other):
            return mycmp(self.obj, other.obj) <= 0
        def __ge__(self, other):
            return mycmp(self.obj, other.obj) >= 0
        def __ne__(self, other):
            return mycmp(self.obj, other.obj) != 0
    return K

In [4]:
q = 7
k = 8
a = vector([5,1,2,3,4,2,1,5])
b = vector([1,2,1,0,6,4,4,2])
matrixList = makePublicKey(q,k)
system, R_ =  algebraicAttackSystem(q,k,a,b, matrixList)


[0 6 5 6 4 3 4 0 6 1 0 3 1 1 3 0]
[6 3 2 2 2 0 5 3 1 6 2 2 2 6 2 4]
[6 3 5 1 5 0 6 0 5 1 0 4 6 3 4 0]
[0 2 5 1 6 0 1 0 1 0 0 4 0 0 4 5]
[3 3 3 5 2 0 3 2 3 4 1 3 6 5 2 4]
[1 5 0 0 0 0 4 1 6 0 4 2 0 2 2 0]
[5 2 5 4 2 1 1 4 5 1 2 5 3 1 0 4]
[2 2 1 6 5 2 1 1 4 6 2 1 6 5 0 6]
[0 3 0 6 6 0 1 5 6 2 0 4 2 1 5 2]
[5 1 4 3 4 5 3 2 4 1 5 2 2 0 3 3]
[0 4 4 6 3 5 0 5 0 6 1 2 0 3 5 4]
[5 3 4 2 4 1 3 6 6 0 2 4 5 4 3 6]
[4 5 0 2 4 4 1 2 4 6 1 0 3 3 6 3]
[6 0 5 5 1 4 1 2 3 6 2 1 2 3 3 4]
[1 1 2 2 5 4 2 6 6 3 3 0 3 4 4 4]
[6 0 1 5 2 2 2 5 3 5 3 5 0 2 4 2]
Defining x0, x1, x2, x3, x4, x5, x6, x7, x8, x9, x10, x11, x12, x13, x14, x15, xn


In [25]:
gb

{(x4^2*xn^2 + x4*x5*xn^2 + x5^2*xn^2, (x4^2, 0)),
 (x4^4*xn^2 - x4^2*x5^2*xn^2 - 2*x4*x5^3*xn^2 - x5^4*xn^2, (x4*x5^3, 0)),
 (x4^5*xn^2 - x4^4*x5*xn^2 - x4^3*x5^2*xn^2 + 2*x4*x5^4*xn^2 + 2*x5^5*xn^2,
  (x5^5, 0)),
 (x4^6*xn^2 + 2*x4^5*x5*xn^2 - 3*x4^4*x5^2*xn^2 - 3*x4^3*x5^3*xn^2 + x4^2*x5^4*xn^2 - x4*x5^5*xn^2 - 2*x5^6*xn^2,
  (x3*x4^5, 1)),
 (x3*xn^4 + x4*xn^4 - 3*x5*xn^4, (x2*x5^2, 0)),
 (x3*x5*xn^2 - 3*x4^2*xn^2 - 2*x4*x5*xn^2 + x5^2*xn^2, (x3*x4, 0)),
 (x3*x5^2*xn^2 - 2*x4^3*xn^2 - 2*x4^2*x5*xn^2 - x4*x5^2*xn^2 - 3*x5^3*xn^2,
  (x4*x5^2, 0)),
 (x3*x5^3*xn^2 - 2*x4^4*xn^2 - 2*x4^3*x5*xn^2 + 3*x4*x5^3*xn^2 - x5^4*xn^2,
  (x3*x4^3, 1)),
 (x3*x4*xn^2 - x3*x5*xn^2 + x4^2*xn^2 + 3*x4*x5*xn^2 + 3*x5^2*xn^2,
  (x4*x5, 0)),
 (x3*x4*x5*xn^2 - 2*x3*x5^2*xn^2 + 2*x4^3*xn^2 + 3*x4^2*x5*xn^2 - 3*x4*x5^2*xn^2 - x5^3*xn^2,
  (x5^3, 0)),
 (x3*x4^2*xn^2 + x3*x4*x5*xn^2 + 3*x3*x5^2*xn^2 - 2*x4^3*xn^2 - 3*x4^2*x5*xn^2 - x4*x5^2*xn^2,
  (x3*x4^2, 1)),
 (x3*x4^3*xn^2 + 3*x3*x4^2*x5*xn^2 - 2*x3*x4*x5^2*

In [5]:
singular.lib('f5_library_new.lib')

In [6]:
R2 = R_._singular_()
i2 = singular.ideal([str(i) for i in system])

In [12]:
R_.gens()

(x0, x1, x2, x3, x4, x5, xn)

In [15]:
R2

polynomial ring, over a field, global ordering
// coefficients: ZZ/7
// number of vars : 7
//        block   1 : ordering dp
//                  : names    x0 x1 x2 x3 x4 x5 xn
//        block   2 : ordering C

In [7]:
a = singular.basis_r(i2)

Interrupting Singular...


KeyboardInterrupt: Restarting Singular (WARNING: all variables defined in previous session are now invalid)

In [19]:
b = a.sage()

In [16]:
a = b.groebner_basis()

In [20]:
b = ideal(b)

In [21]:
b.groebner_basis()

Polynomial Sequence with 26 Polynomials in 7 Variables

In [22]:
I = ideal(system)

In [24]:
I = I.subs(xn = 1)

In [25]:
I.groebner_basis()

[x1*x5^2 + 2*x4 + 2*x5, x1^2 + x1*x2 + x2^2, x1*x4 - 2, x2*x4 + x1*x5 + 2, x4^2 + x4*x5 + x5^2, x2*x5 - 2, x0 + x1 - 3*x2, x3 + x4 - 3*x5]

-3*x1*x3-3*x2*x3-3*x0*x4-x1*x4-3*x2*x4-3*x0*x5-3*x1*x5-3*x2*x5,
2*x0*x3-x1*x3-x0*x4-3*x2*x5,
-2*x0*x3-3*x2*x3-2*x1*x4-2*x2*x4-3*x0*x5-2*x1*x5+2*x2*x5-1,
3*x0*x3-2*x2*x3+2*x1*x4-x2*x4-2*x0*x5-x1*x5+2*x2*x5+3,
x0*x3-x1*x3-x2*x3-x0*x4+3*x1*x4-x0*x5+x2*x5-2,
x0*x3-3*x1*x3+2*x2*x3-3*x0*x4+3*x1*x4-2*x2*x4+2*x0*x5-2*x1*x5+x2*x5+2